In [4]:
# import kagglehub
# custom_path = r"C:\Users\Tanuj Rajput\OneDrive\Desktop\tanujkaggle\12"
# path = kagglehub.dataset_download("saurabhshahane/fake-news-classification")

# import shutil
# shutil.move(path, custom_path)

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv("WELFake_Dataset.csv")

In [7]:
df = df.drop(columns=["Unnamed: 0"])

In [8]:
df['content'] = df['title'] + " " + df['text']

In [9]:
df.dropna(subset=['content', 'label'], inplace=True)

In [10]:
df

title  \
0      LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
2      UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3      Bobby Jindal, raised Hindu, uses story of Chri...   
4      SATAN 2: Russia unvelis an image of its terrif...   
5      About Time! Christian Group Sues Amazon and SP...   
...                                                  ...   
72129  Russians steal research on Trump in hack of U....   
72130   WATCH: Giuliani Demands That Democrats Apolog...   
72131  Migrants Refuse To Leave Train At Refugee Camp...   
72132  Trump tussle gives unpopular Mexican leader mu...   
72133  Goldman Sachs Endorses Hillary Clinton For Pre...   

                                                    text  label  \
0      No comment is expected from Barack Obama Membe...      1   
2       Now, most of the demonstrators gathered last ...      1   
3      A dozen politically active pastors came here f...      0   
4      The RS-28 Sarmat missile, dubbed Satan 2, will...      1   
5      All we can say on this one is it s about time ...      1   
...                                                  ...    ...   
72129  WASHINGTON (Reuters) - Hackers believed to be ...      0   
72130  You know, because in fantasyland Republicans n...      1   
72131  Migrants Refuse To Leave Train At Refugee Camp...      0   
72132  MEXICO CITY (Reuters) - Donald Trump’s combati...      0   
72133  Goldman Sachs Endorses Hillary Clinton For Pre...      1   

                                                 content  
0      LAW ENFORCEMENT ON HIGH ALERT Following Threat...  
2      UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...  
3      Bobby Jindal, raised Hindu, uses story of Chri...  
4      SATAN 2: Russia unvelis an image of its terrif...  
5      About Time! Christian Group Sues Amazon and SP...  
...                                                  ...  
72129  Russians steal research on Trump in hack of U....  
72130   WATCH: Giuliani Demands That Democrats Apolog...  
72131  Migrants Refuse To Leave Train At Refugee Camp...  
72132  Trump tussle gives unpopular Mexican leader mu...  
72133  Goldman Sachs Endorses Hillary Clinton For Pre...  

[71537 rows x 4 columns]

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [12]:
tokenizer = Tokenizer(num_words=5000, oov_token="<OOV>")
tokenizer.fit_on_texts(df['content'])

In [13]:
sequences = tokenizer.texts_to_sequences(df['content'])
padded = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')

In [14]:
labels = df['label'].values

In [15]:
x_train, x_test, y_train, y_test = train_test_split(padded, labels, test_size=0.2, random_state=42)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout

In [17]:
model = Sequential([
    Embedding(input_dim=5000, output_dim=64, input_length=100),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(64),
    Dropout(0.3),
    Dense(32, activation='relu'),  
    Dense(1, activation='sigmoid')
])

C:\Users\Tanuj Rajput\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])   
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
history = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data=(x_test, y_test))

Epoch 1/5
895/895 ━━━━━━━━━━━━━━━━━━━━ 110s 119ms/step - accuracy: 0.8107 - loss: 0.4198 - val_accuracy: 0.8275 - val_loss: 0.4528
Epoch 2/5
895/895 ━━━━━━━━━━━━━━━━━━━━ 109s 122ms/step - accuracy: 0.7763 - loss: 0.4917 - val_accuracy: 0.6443 - val_loss: 0.6188
Epoch 3/5
895/895 ━━━━━━━━━━━━━━━━━━━━ 110s 123ms/step - accuracy: 0.6503 - loss: 0.6124 - val_accuracy: 0.6108 - val_loss: 0.6373
Epoch 4/5
895/895 ━━━━━━━━━━━━━━━━━━━━ 110s 123ms/step - accuracy: 0.7213 - loss: 0.5438 - val_accuracy: 0.9331 - val_loss: 0.1870
Epoch 5/5
895/895 ━━━━━━━━━━━━━━━━━━━━ 111s 124ms/step - accuracy: 0.9470 - loss: 0.1572 - val_accuracy: 0.9593 - val_loss: 0.1076


In [21]:
import pickle

In [23]:
model.save("fake_news_classifier_model.h5")

In [30]:
with open('tokenizer.pkl', 'wb') as f:      
    pickle.dump(tokenizer, f)
print("Model and tokenizer saved successfully.")

Model and tokenizer saved successfully.


In [31]:
from tensorflow.keras.models import load_model

In [32]:
model = load_model("fake_news_classifier_model.h5")

In [33]:
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [34]:
def predict_news(title, text):
    content = title + " " + text
    seq = tokenizer.texts_to_sequences([content])
    padded_seq = pad_sequences(seq, maxlen=500, padding='post', truncating='post') 
    pred = model.predict(padded_seq)[0][0]
    return "Real" if pred > 0.5 else "Fake"

In [38]:
!python -m pip install streamlit

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-auth 2.39.0 requires pyasn1-modules>=0.2.1, which is not installed.
langchain-community 0.3.21 requires aiohttp<4.0.0,>=3.8.3, which is not installed.
langchain-community 0.3.21 requires pydantic-settings<3.0.0,>=2.4.0, which is not installed.
langchain-core 0.3.51 requires jsonpatch<2.0,>=1.33, which is not installed.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/9.9 MB 2.8 MB/s eta 0:00:04
   ---- ----------------------------------- 1.0/9.9 MB 3.1 MB/s eta 0:00:03
   ------ --------------------------------- 1.6/9.9 MB 2.6 MB/s eta 0:00:04
   -------- ------------------------------- 2.1/9.9 MB 2.5 MB/s eta 0:00:04
   ----------- ---------------------------- 2.9/9.9 MB 2.7 MB/s eta 0:00:03
   ------------- -------------------------- 3.4/9.9 MB 2.8 MB/s eta 0:00:03
   ----------------- ---------------------- 4.2/9.9 MB 2.9 MB/s eta 0:00:02
   ------------------- -------------------- 4.7/9.9 MB 2.9 MB/s eta 0:00:02
   ---------------------- ----------------- 5.5/9.9 MB 2.9 MB/s eta 0:00:02
   ----------------------- ---------------- 5.8/9.9 MB 2.9 MB/s eta 0:00:02
   ------------------------- ----------